In [2]:
''' Solve the Alignment with Affine Gap Penalties Problem.'''

import pandas as pd
BLOSUM62=pd.read_table('BLOSUM62.txt', delim_whitespace=True,index_col=0)
BLOSUM62['-']=-1
BLOSUM62.loc['-'] = -1
BLOSUM62['-']['-']=''
#print BLOSUM62


def LCSBackTrack(v, w):
    import numpy as np
    
    ### initializing tables ###
    sigma=-11 #gap penalty
    lower = np.full((len(v)+1,len(w)+1),-np.inf)
    upper = np.full((len(v)+1,len(w)+1),-np.inf)
    middle = np.zeros((len(v)+1,len(w)+1))
    btlower = np.empty([len(v)+1, len(w)+1], dtype = str)
    btupper = np.empty([len(v)+1, len(w)+1], dtype = str)
    btmiddle = np.empty([len(v)+1, len(w)+1], dtype = str)
    
    lower[0,0] = 0
    upper[0,0] = 0
    lower[1,0] = sigma
    upper[0,1] = sigma
    for i in range(2, len(v)+1):
        lower[i,0]=lower[i-1,0]+BLOSUM62[v[i-1]]['-']
    for j in range(2, len(w)+1):
        upper[0,j]=upper[0,j-1]+BLOSUM62['-'][w[j-1]]
    
    ### dynamic scoring ###
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            ## lower
            values=[lower[i-1,j]+BLOSUM62[v[i-1]]['-'] , middle[i-1,j]+sigma]
            lower[i,j] = max(values)
            btlower[i,j] = values.index(lower[i,j])
            
            ## upper 
            values=[upper[i,j-1]+BLOSUM62['-'][w[j-1]], middle[i,j-1]+sigma]
            upper[i,j] = max(values)
            btupper[i,j] = values.index(upper[i,j])
            
            ## middle
            values=[middle[i-1,j-1]+BLOSUM62[v[i-1]][w[j-1]],lower[i,j], upper[i,j]]
            middle[i,j] = max(values)
            btmiddle[i,j] = values.index(middle[i,j])

    return middle[i,j], btlower, btupper,btmiddle


def OutputLCS(v,w):
    score, btlower, btupper, btmiddle =LCSBackTrack(v, w)
    i=len(v)
    j=len(w)
    LCS_v=''
    LCS_w=''
    pointer='middle'
    while i>0 and j>0:
        if pointer == 'middle':
            btmatrix=btmiddle[:]
            if btmatrix[i,j]=='0':
                LCS_v+=v[i-1]
                LCS_w+=w[j-1]
                i = i-1
                j = j-1
                continue
            elif btmatrix[i,j] == '1':
                pointer = 'lower'
                continue
            elif btmatrix[i,j] == '2':
                pointer = 'upper'
                continue
            continue
        
        if pointer == 'lower':
            btmatrix=btlower[:]
            LCS_v+=v[i-1]
            LCS_w+='-'
            i = i-1
            if btmatrix[i,j] == '1':
                pointer ='middle'
                continue
            continue
   
        if pointer == 'upper':
            btmatrix=btupper[:]
            LCS_v+='-'
            LCS_w+=w[j-1]
            j = j-1
            if btmatrix[i,j] == '1':
                pointer ='middle'
                continue
            break
                
    
    while i==0 and j>0 or i>0 and j==0:
        if i==0 and j>0:
            LCS_v+='-'
            LCS_w+=w[j-1]
            j=j-1
            continue
        if i>0 and j==0:
            LCS_v+=v[i-1]
            LCS_w+='-' 
            i=i-1
            continue

    
    return score,LCS_v[::-1],LCS_w[::-1]

#### TEST 1 : worked ####

v='PRTEINS'
w='PRTWPSEIN'


#### TEST 2  ####
'''
f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]
'''

#### test 3 :  work ####
'''
v='AHRQPQ'
w='AHED'
'''
#### test 4 : worked ####
'''
v='GCYTVVK'
w='DVVK'
'''
#### CHALLENGE : passed ####

#f=open('../../Downloads/dataset_249_8 (1).txt','r')
# f=open('../../Downloads/rosalind_ba5j.txt','r')
# lines=f.read().splitlines()
# f.close()
# v=lines[0]
# w=lines[1]


import time
start_time = time.time()

#print '\n'.join(map(str,LCSBackTrack(v, w)))
print '\n'.join(map(str,OutputLCS(v,w)))
print ('The script took '+str(time.time()-start_time)+' s')

8.0
-EINS
SEIN-
The script took 0.00667309761047 s


In [48]:
''' Solve the Middle Edge in Linear Space Problem (for protein strings).'''


import pandas as pd
BLOSUM62=pd.read_table('BLOSUM62.txt', delim_whitespace=True,index_col=0)
BLOSUM62['-']=-5
BLOSUM62.loc['-'] = -5
BLOSUM62['-']['-']=''
#print matrix['L']['M'],matrix['M']['L']


def LCSBackTrack(v, w):
    import numpy as np
    s = np.zeros((len(v)+1,len(w)+1))
    col_middle = len(w)/2
    for i in range(1, len(v)+1):
        s[i,0]=s[i-1,0]+BLOSUM62[v[i-1]]['-']
    for j in range(1, len(w)+1): 
        s[0,j]=s[0,j-1]+BLOSUM62['-'][w[j-1]]
    
    for i in range(1,len(v)+1):
        for j in range(1,col_middle+2):
            s[i,j] = max(s[i-1,j]+BLOSUM62[v[i-1]]['-'] , s[i,j-1]+BLOSUM62['-'][w[j-1]], s[i-1,j-1]+BLOSUM62[v[i-1]][w[j-1]])
    row_middle=np.argmax(s[:,col_middle])
    row_edge=(s[row_middle,col_middle+1],s[row_middle+1,col_middle+1])
    if s[row_middle,col_middle+1]>s[row_middle+1,col_middle+1]:
        row_edge= row_middle
    else:
        row_edge=row_middle+1
    col_edge=col_middle+1
    return (row_middle,col_middle),(row_edge,col_edge)


#### test 1 :passed ####

v='PLEASANTLY'
w='MEASNLY'

#### test 2 : passed ####

f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]

#### challenge : passed ####

#f=open('../../Downloads/dataset_250_12.txt','r')
f=open('../../Downloads/rosalind_ba5k (1).txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]

print LCSBackTrack(v,w)

((512, 524), (513, 525))


In [22]:
import pandas as pd
BLOSUM62=pd.read_table('BLOSUM62.txt', delim_whitespace=True,index_col=0)
#BLOSUM62['-']=-5
#BLOSUM62.loc['-'] = -5
#BLOSUM62['-']['-']=''
#print BLOSUM62
import numpy as np

def MiddleNode_and_Edge(top,bottom,left,right):
    import numpy as np
    col_middle=int(round(float(left)+float(right))/2)
    gap=-5
    #####from source#####
    
     #v2#
    fromsource=np.zeros(((bottom-top)+1,2))
    for i in range(1,len(fromsource)):
        fromsource[i,1]=fromsource[i-1,1]+gap
    windex=left
    while windex <col_middle:
        empty = np.zeros(((bottom-top)+1,2))
        empty[:,0] = fromsource[:,1]
        fromsource=empty[:]
        for i in range(len(empty)):
            if i==0:
                fromsource[i,1]=fromsource[i,0]+gap
            else:
                fromsource[i,1] = max(fromsource[i-1,1]+gap , fromsource[i,0]+gap, fromsource[i-1,0]+BLOSUM62[v[top+i-1]][w[windex]])
        windex+=1 
    row_middle_1=fromsource[:,1]

    #V1#
    '''
    fromsource=s.copy()
    for i in range(top+1, bottom+1):
        fromsource[i,left]=fromsource[i-1,left]+BLOSUM62[v[i-1]]['-']
    for j in range(left+1, col_middle+1): 
        fromsource[top,j]=fromsource[top,j-1]+BLOSUM62['-'][w[j-1]]
    for i in range(top+1,bottom+1):
        for j in range(left+1,col_middle+1):
            fromsource[i,j] = max(fromsource[i-1,j]+BLOSUM62[v[i-1]]['-'] , fromsource[i,j-1]+BLOSUM62['-'][w[j-1]], fromsource[i-1,j-1]+BLOSUM62[v[i-1]][w[j-1]])
    print fromsource
    row_middle_1=fromsource[top:bottom+1,col_middle]
    print row_middle_1
    '''
    #####tosink####
    
    ##V2 ##
    
    tosink=np.zeros(((bottom-top)+1,2))
    for i in range(len(tosink)-2,-1,-1):
        tosink[i,0]=tosink[i+1,0]+gap
    windex=right
    while windex > col_middle:
        empty = np.zeros(((bottom-top)+1,2))
        empty[:,1] = tosink[:,0]
        tosink=empty[:]
        for i in range(len(tosink)-1,-1,-1):
            if i==len(tosink)-1:
                tosink[i,0]=tosink[i,1]+gap
            else:
                tosink[i,0] = max(tosink[i+1,0]+gap , tosink[i,1]+gap, tosink[i+1,1]+BLOSUM62[v[top+i]][w[windex-1]])
        windex=windex-1
    row_middle_2=tosink[:,0]    
    #print tosink
    #print row_middle_2
    ## V1##
    '''
    
    tosink=s.copy()
    for i in range(bottom-1,top-1,-1):
        tosink[i,right]=tosink[i+1,right]+BLOSUM62[v[i]]['-']
    for j in range(right-1,col_middle-1,-1): 
        tosink[bottom,j]=tosink[bottom,j+1]+BLOSUM62['-'][w[j]]
    #print tosink    
    #print col_middle
    for i in range(bottom-1,top-1,-1):
        for j in range(right-1,col_middle-1,-1):
            tosink[i,j] = max(tosink[i+1,j]+BLOSUM62[v[i-1]]['-'] , tosink[i,j+1]+BLOSUM62['-'][w[j-1]], tosink[i+1,j+1]+BLOSUM62[v[i]][w[j]])
            #print v[i],[w[j]]

    row_middle_2=tosink[top:bottom+1,col_middle]
    #print row_middle_2
    print tosink
    '''
   
    ##### middleEdge #####
    
    row_middle_subset=row_middle_1+row_middle_2
    max_score=row_middle_subset.max()
    index_max_subset=np.where(row_middle_subset == max_score)[0][-1]
    row_midNode=index_max_subset+top
    #print tosink
    #print row_middle_subset,max_score,index_max_subset
    #print row_midNode, col_middle
    try:
    #print tosink[index_max_subset+1,1],tosink[index_max_subset,0],BLOSUM62[v[row_midNode]][w[col_middle]]
        if tosink[index_max_subset+1,1]==tosink[index_max_subset,0]-BLOSUM62[v[row_midNode]][w[col_middle]]:
            midEdge='T'
        else:
            midEdge='R'
    except IndexError :
        midEdge='R'
    return max_score,row_midNode,midEdge



def LinearSpaceAlignment(v, w, top, bottom, left, right):
    i=0
    import numpy as np
    ### init ###       
    ### divide and conquer ###    
    if left == right:
        return (bottom-top)*'D'
        #output path formed by bottom − top vertical edges
    elif top == bottom:
        return (right-left)*'R'
        #output path formed by right − left horizontal edges
    else:    
        middle=int(round(float(left)+float(right))/2)
        score,midNode,midEdge = MiddleNode_and_Edge(top, bottom, left, right)
        up_bottom=midNode
        up_right= middle
        if midEdge == "R" or midEdge == "T":
            down_left = middle + 1
        else:
            down_left=middle
            
        if midEdge == "D" or midEdge =="T":
            down_top =midNode + 1
        else:
            down_top=midNode
        return LinearSpaceAlignment(v,w,top, up_bottom, left, up_right) +str(midEdge) + LinearSpaceAlignment(v,w,down_top, bottom, down_left, right)
                                                                                                                      
def Align_from_path(v,w,path):
    LCS_v=''
    LCS_w=''
    i=0
    j=0
    for n in path:
        if n=='D':
            LCS_v+=v[i]
            LCS_w+='-'
            i+=1
        elif n=='R':
            LCS_v+='-'
            LCS_w+=w[j]
            j+=1
        elif n=='T':
            LCS_v+=v[i]
            LCS_w+=w[j]
            i+=1
            j+=1
    return LCS_v,LCS_w

def score(v,w,matrix):
    score=0
    for i in range(len(v)):
        score+=matrix[v[i]][w[i]]
    return score    
            
#### TEST 1 :worked ####   

#v='PLEASANTLY'
#w='MEANLY'

#### test 2: worked ####
    
#v = "PRTEINS"
#w = "PRTWPSEIN"

### test ####

#v='AHED'
#w='AHRQPQ'
#### test 3 ####
'''
f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]
'''
#### CHALLENGE ####

#f=open('../../Downloads/dataset_250_14.txt','r')
f=open('../../Downloads/rosalind_ba5e (2).txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]

top=0
left=0
bottom=len(v)
right=len(w)

#s = np.zeros((len(v)+1,len(w)+1))
#print MiddleNode_and_Edge(s,top,bottom,left,right)
#print MiddleNode_and_Edge(s,0, 6, 0, 3)
#print MiddleNode_and_Edge(7, 10, 4, 6)
#print MiddleNode_and_Edge(s,7, 2, 4, 5)
#print MiddleNode_and_Edge(s,3, 4, 5, 7)
#print LinearSpaceAlignment(v, w, top, bottom, left, right)
import time
start_time = time.time()

path=LinearSpaceAlignment(v, w, top, bottom, left, right)
alignment=Align_from_path(v,w,path)
score=MiddleNode_and_Edge(top,bottom,left,right)[0]
print score
print '\n'.join(alignment)
print ('The script took '+str(time.time()-start_time)+' s')

10826.0
TCYPALQYM---ARHSNHLFYAMVHTLSDKGLDIIEVGVLREVRLAYCGISCRPNMSLIQTQDWPHTGHEWIERMCNTR-TCTYHCRTQ--DWS-SMCIMVVWM----V-TFC-HS-----DFYQTMPLHQTV-EISDGFCMSWRKLITDEQKKLYAVLYYNLIELTQSNW-SDNTMFEIHTHKNGRMTEEVIMTRDQFLRSKCDKIRWMWYRWMPDTAFYECHGRLTAHDDGHVEQNNSGKSFEVY-QIYVMGVQLKCQVPQPHAWTIMDAASKVFDVLTVWSACVEYKFKTQMQRGWALVSFCHTYSECFEWMSVKLTNEHVADKCAQELWHFCTN-ALLPCCMHQGSTLTREDVWTSM--------WLKDFVNRRIAIGQ--P--P-EAAEMMMWWNCC---KIHGADAFSDCRTVHCNIVSGIEMAPLKVKHEDQEWPECDLHTNTLHPVQLWCRTPFPDQGGCG-LSKTPAIT-------PIDECQRAKPMKTLHHRDNMGE---SNNMDCTFPDSFINGYHLPKMKPTPTAEKASFP-CGRVCTRAHFWMETGVILMHFHFRHQKRKGHHAEVFCMVCDAMDANELWQT-D-M---PVELEMIMASNVRAIALRYWFGPTRYNAPCSKHNGVWCHVAMPNSARQCDARIITWITHNVCMPKH-LSSLW-YCEPEESPTYEAPQTSIFASLTDIELTFLWIKATGPIIP-IMCR-RFFTMNHLKKI-Q--GWC-----WRNGHMYNVNQWQFHEKY-EGLTKL---T----AT---LCPLGVHVDYSNDDVVCCKFTSMMPQTIWWEMNTAHAIPLTEPQTMCVDPNFWFAHHYSHAGCAYWWKDVSYNIWYRMFGDISDTASIIYFYNEQTHREANEFPRTYYAHINQSPTTAEEEMKRFESSIVWKKCYSSLPSMCYVWLCQVPPVFHRIRATIGPERWETMDRTGSRLDWTETKPEQLHHFWFK--L-W--PCNEESFQKDHEG

In [208]:
import pandas as pd
BLOSUM62=pd.read_table('BLOSUM62.txt', delim_whitespace=True,index_col=0)
BLOSUM62['-']=-5
BLOSUM62.loc['-'] = -5
BLOSUM62['-']['-']=''
#print BLOSUM62
import numpy as np

def MiddleNode_and_Edge(top,bottom,left,right):
    import numpy as np
    col_middle=int(round(float(left)+float(right))/2)
    
    #####from source#####
    
     #v2#
    fromsource=np.zeros(((bottom-top)+1,2))
    for i in range(1,len(fromsource)):
        fromsource[i,1]=fromsource[i-1,1]+BLOSUM62[v[top+i-1]]['-']
    windex=left
    while windex <col_middle:
        empty = np.zeros(((bottom-top)+1,2))
        empty[:,0] = fromsource[:,1]
        fromsource=empty[:]
        for i in range(len(empty)):
            j=1
            if i==0:
                fromsource[i,j]=fromsource[i,j-1]+BLOSUM62['-'][w[windex]]
            else:
                fromsource[i,j] = max(fromsource[i-1,j]+BLOSUM62[v[top+i-1]]['-'] , fromsource[i,j-1]+BLOSUM62['-'][w[windex+j-1]], fromsource[i-1,j-1]+BLOSUM62[v[top+i-1]][w[windex+j-1]])
        windex+=1 
    row_middle_1=fromsource[:,1]

    #####tosink####
    
    ##V2 ##
    
    tosink=np.zeros(((bottom-top)+1,2))
    for i in range(len(tosink)-2,-1,-1):
        tosink[i,0]=tosink[i+1,0]+BLOSUM62[v[top+i]]['-']
    windex=right
    while windex > col_middle:
        empty = np.zeros(((bottom-top)+1,2))
        empty[:,1] = tosink[:,0]
        tosink=empty[:]
        for i in range(len(tosink)-1,-1,-1):
            j=0
            if i==len(tosink)-1:
                tosink[i,j]=tosink[i,j+1]+BLOSUM62['-'][w[windex-1]]
            else:
                tosink[i,j] = max(tosink[i+1,0]+BLOSUM62[v[top+i]]['-'] , tosink[i,1]+BLOSUM62['-'][w[windex-1]], tosink[i+1,j+1]+BLOSUM62[v[top+i]][w[windex-1]])
        windex=windex-1
    row_middle_2=tosink[:,0]    
    
    ##### middleEdge #####
    
    row_middle_subset=row_middle_1+row_middle_2
    index_max_subset=np.where(row_middle_subset == row_middle_subset.max())[0][-1]
    row_midNode=index_max_subset+top
    #print index_max_subset,row_middle_subset
    if tosink[index_max_subset+1,1]==tosink[index_max_subset,0]-BLOSUM62[v[row_midNode]][w[col_middle]]:
        midEdge='T'
    else:
        midEdge='R'
    return row_midNode,midEdge



def LinearSpaceAlignment(v, w, top, bottom, left, right):
    import numpy as np
    ### init ###       
    ### divide and conquer ###    
    if left == right:
        return (bottom-top)*'D'
        #output path formed by bottom − top vertical edges
    elif top == bottom:
        return (right-left)*'R'
        #output path formed by right − left horizontal edges
    else:    
        middle=int(round(float(left)+float(right))/2)
        midNode,midEdge = MiddleNode_and_Edge(top, bottom, left, right)
        up_bottom=midNode
        up_right= middle
        if midEdge == "R" or midEdge == "T":
            down_left = middle + 1
        else:
            down_left=middle
            
        if midEdge == "D" or midEdge =="T":
            down_top =midNode + 1
        else:
            down_top=midNode
        return LinearSpaceAlignment(v,w,top, up_bottom, left, up_right) +str(midEdge) + LinearSpaceAlignment(v,w,down_top, bottom, down_left, right)
                                                                                                                      
def Align_from_path(v,w,path):
    LCS_v=''
    LCS_w=''
    i=0
    j=0
    for n in path:
        if n=='D':
            LCS_v+=v[i]
            LCS_w+='-'
            i+=1
        elif n=='R':
            LCS_v+='-'
            LCS_w+=w[j]
            j+=1
        elif n=='T':
            LCS_v+=v[i]
            LCS_w+=w[j]
            i+=1
            j+=1
    return LCS_v,LCS_w

def score(v,w,matrix):
    score=0
    for i in range(len(v)):
        score+=matrix[v[i]][w[i]]
    return score    
            
#### TEST 1 :worked ####   

#v='PLEASANTLY'
#w='MEANLY'

#### test 2: worked ####
    
#v = "PRTEINS"
#w = "PRTWPSEIN"

#### test 3 ####
'''
f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]
'''
#### CHALLENGE ####

f=open('../../Downloads/dataset_250_14.txt','r')
#f=open('../../Downloads/rosalind_ba5e (1).txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]


top=0
left=0
bottom=len(v)
right=len(w)

#s = np.zeros((len(v)+1,len(w)+1))
#print MiddleNode_and_Edge(s,top,bottom,left,right)
#print MiddleNode_and_Edge(s,0, 6, 0, 3)
#print MiddleNode_and_Edge(7, 10, 4, 6)
#print MiddleNode_and_Edge(s,7, 2, 4, 5)
#print MiddleNode_and_Edge(s,3, 4, 5, 7)
#print LinearSpaceAlignment(v, w, top, bottom, left, right)
import time
start_time = time.time()

path=LinearSpaceAlignment(v, w, top, bottom, left, right)
alignment=Align_from_path(v,w,path)
score=score(str(alignment[0]),str(alignment[1]),BLOSUM62)
print score
print '\n'.join(alignment)
print ('The script took '+str(time.time()-start_time)+' s')

KeyboardInterrupt: 

In [202]:
import pandas as pd
BLOSUM62=pd.read_table('BLOSUM62.txt', delim_whitespace=True,index_col=0)
BLOSUM62['-']=-5
BLOSUM62.loc['-'] = -5
BLOSUM62['-']['-']=''
#print BLOSUM62
import numpy as np

def MiddleNode_and_Edge(s,top,bottom,left,right):
    import numpy as np
    col_middle=int(round(float(left)+float(right))/2)
    
    #####from source#####

    #V1#
   
    fromsource=s.copy()
    for i in range(top+1, bottom+1):
        fromsource[i,left]=fromsource[i-1,left]+BLOSUM62[v[i-1]]['-']
    for j in range(left+1, col_middle+1): 
        fromsource[top,j]=fromsource[top,j-1]+BLOSUM62['-'][w[j-1]]
    for i in range(top+1,bottom+1):
        for j in range(left+1,col_middle+1):
            fromsource[i,j] = max(fromsource[i-1,j]+BLOSUM62[v[i-1]]['-'] , fromsource[i,j-1]+BLOSUM62['-'][w[j-1]], fromsource[i-1,j-1]+BLOSUM62[v[i-1]][w[j-1]])
    #print fromsource
    row_middle_1=fromsource[top:bottom+1,col_middle]
    #print row_middle_1
    
    #####tosink####
   
    ## V1##
    
    
    tosink=s.copy()
    for i in range(bottom-1,top-1,-1):
        tosink[i,right]=tosink[i+1,right]+BLOSUM62[v[i]]['-']
    for j in range(right-1,col_middle-1,-1): 
        tosink[bottom,j]=tosink[bottom,j+1]+BLOSUM62['-'][w[j]]
    #print tosink    
    #print col_middle
    for i in range(bottom,top-1,-1):
        for j in range(right-1,col_middle-1,-1):
            tosink[i,j] = max(tosink[i+1,j]+BLOSUM62[v[i-1]]['-'] , tosink[i,j+1]+BLOSUM62['-'][w[j-1]], tosink[i+1,j+1]+BLOSUM62[v[i]][w[j]])
            #print v[i],[w[j]]

    row_middle_2=tosink[top:bottom+1,col_middle]
    
    ##### middleEdge #####
    
    row_middle_subset=row_middle_1+row_middle_2
    index_max_subset=np.where(row_middle_subset == row_middle_subset.max())[0][-1]
    row_midNode=index_max_subset+top

    if tosink[row_midNode+1,col_middle+1]==tosink[row_midNode,col_middle]-BLOSUM62[v[row_midNode]][w[col_middle]]:
        midEdge='T'
    else:
        midEdge='R'
    return row_midNode,midEdge



def LinearSpaceAlignment(v, w, top, bottom, left, right):
    i=0
    import numpy as np
    ### init ###       
    ### divide and conquer ###    
    if left == right:
        return (bottom-top)*'D'
        #output path formed by bottom − top vertical edges
    elif top == bottom:
        return (right-left)*'R'
        #output path formed by right − left horizontal edges
    else:    
        middle=int(round(float(left)+float(right))/2)
        s=np.zeros((len(v)+1,len(w)+1))
        midNode,midEdge = MiddleNode_and_Edge(s,top, bottom, left, right)
        up_bottom=midNode
        up_right= middle
        if midEdge == "R" or midEdge == "T":
            down_left = middle + 1
        else:
            down_left=middle
            
        if midEdge == "D" or midEdge =="T":
            down_top =midNode + 1
        else:
            down_top=midNode
        return LinearSpaceAlignment(v,w,top, up_bottom, left, up_right) +str(midEdge) + LinearSpaceAlignment(v,w,down_top, bottom, down_left, right)
                                                                                                                      
def Align_from_path(v,w,path):
    LCS_v=''
    LCS_w=''
    i=0
    j=0
    for n in path:
        if n=='D':
            LCS_v+=v[i]
            LCS_w+='-'
            i+=1
        elif n=='R':
            LCS_v+='-'
            LCS_w+=w[j]
            j+=1
        elif n=='T':
            LCS_v+=v[i]
            LCS_w+=w[j]
            i+=1
            j+=1
    return LCS_v,LCS_w

def score(v,w,matrix):
    score=0
    for i in range(len(v)):
        score+=matrix[v[i]][w[i]]
    return score    
            
#### TEST 1 :worked ####   

#v='PLEASANTLY'
#w='MEANLY'

#### test 2: worked ####
    
v = "PRTEINS"
w = "PRTWPSEIN"

#### test 3 ####
'''
f=open('input.txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]
'''
#### CHALLENGE ####
'''
#f=open('../../Downloads/dataset_247_3.txt','r')
f=open('../../Downloads/rosalind_ba5e (1).txt','r')
lines=f.read().splitlines()
f.close()
v=lines[0]
w=lines[1]
'''

top=0
left=0
bottom=len(v)
right=len(w)

#s = np.zeros((len(v)+1,len(w)+1))
#print MiddleNode_and_Edge(s,top,bottom,left,right)
#print MiddleNode_and_Edge(s,0, 6, 0, 3)
#print MiddleNode_and_Edge(7, 10, 4, 6)
#print MiddleNode_and_Edge(s,7, 2, 4, 5)
#print MiddleNode_and_Edge(s,3, 4, 5, 7)
#print LinearSpaceAlignment(v, w, top, bottom, left, right)
import time
start_time = time.time()

path=LinearSpaceAlignment(v, w, top, bottom, left, right)
alignment=Align_from_path(v,w,path)
score=score(str(alignment[0]),str(alignment[1]),BLOSUM62)
print score
print '\n'.join(alignment)
print ('The script took '+str(time.time()-start_time)+' s')

IndexError: index 8 is out of bounds for axis 0 with size 8

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


[3, 2, 1, 0]